# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tsiombe,-25.3000,45.4833,21.21,67,65,7.19,MG,1688021690
1,1,blackmans bay,-43.0167,147.3167,8.22,90,100,5.90,AU,1688021691
2,2,bredasdorp,-34.5322,20.0403,12.99,80,90,6.86,ZA,1688021691
3,3,haapsalu,58.9431,23.5414,23.99,69,9,2.80,EE,1688021691
4,4,kerman,30.2832,57.0788,35.07,20,0,4.12,IR,1688021449


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [68]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles = "OSM",
    frame_height=500,
    frame_width=700,
    size='Humidity',
    scale=0.5,
    color='City'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_data_df = city_data_df[(city_data_df["Max Temp"] < 27) 
                                     & (city_data_df["Max Temp"] > 21) 
                                     & (city_data_df["Cloudiness"] ==0) 
                                     & (city_data_df["Wind Speed"] < 4.5)]

# Drop any rows with null values
narrowed_city_data_df = narrowed_city_data_df.dropna()

# Display sample data
narrowed_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,avarua,-21.2078,-159.7750,24.18,100,0,2.06,CK,1688021704
57,57,ribeira brava,32.6500,-17.0667,25.55,51,0,0.35,PT,1688021709
63,63,morondava,-20.2833,44.2833,24.09,39,0,2.32,MG,1688021711
78,78,greenville,35.6127,-77.3663,21.33,92,0,0.00,US,1688021510
93,93,edgewater,28.9889,-80.9023,26.31,89,0,1.23,US,1688021720


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
42,avarua,CK,-21.2078,-159.7750,100,
57,ribeira brava,PT,32.6500,-17.0667,51,
63,morondava,MG,-20.2833,44.2833,39,
78,greenville,US,35.6127,-77.3663,92,
93,edgewater,US,28.9889,-80.9023,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [65]:
# Set parameters to search for a hotel
limit = 200
radius = 10000
params = {
    "limit":limit,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["categories"] = f"accommodation.hotel"
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
avarua - nearest hotel: Paradise Inn
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
morondava - nearest hotel: Hotel Sharon Menabe
greenville - nearest hotel: Quality Inn
edgewater - nearest hotel: Best Western Edgewater Inn
benghazi - nearest hotel: ترجاب
mwinilunga - nearest hotel: No hotel found
sirte - nearest hotel: فندق المدينة-سرت
kasongo-lunda - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
betioky - nearest hotel: Hotel Salema
portland - nearest hotel: the Hoxton
naantali - nearest hotel: Hotel Bridget Inn
pskov - nearest hotel: Октябрьская
douglas - nearest hotel: Residence Inn Omaha West
hukumati azrah - nearest hotel: No hotel found
thomson - nearest hotel: Days Inn by Wyndham Thomson
dixiana - nearest hotel: Courtyard
nova vicosa - nearest hotel: Pousada Pontal da Barra
puerto baquerizo moreno - nearest hotel: Dory's House
remire-montjoly - nearest hotel: Complexe Belova
walterboro - nearest hotel: Travelodg

,City,Country,Lat,Lng,Humidity,Hotel Name
42,avarua,CK,-21.2078,-159.7750,100,Paradise Inn
57,ribeira brava,PT,32.6500,-17.0667,51,Cheerfulway Bravamar Hotel
63,morondava,MG,-20.2833,44.2833,39,Hotel Sharon Menabe
78,greenville,US,35.6127,-77.3663,92,Quality Inn
93,edgewater,US,28.9889,-80.9023,89,Best Western Edgewater Inn
98,benghazi,LY,32.1167,20.0667,68,ترجاب
107,mwinilunga,ZM,-11.7358,24.4293,43,No hotel found
131,sirte,LY,31.2089,16.5887,59,فندق المدينة-سرت
146,kasongo-lunda,CD,-6.4667,16.8167,63,No hotel found
183,fort bragg,US,35.1390,-79.0060,89,Airborne Inn Lodging


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [67]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles = "OSM",
    frame_height=500,
    frame_width=700,
    size='Humidity',
    scale=0.5,
    color='City',
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)